## Análisis y Predicción de Ventas en una Tienda de Retail (Core)

# Parte III: Transformación y Análisis Avanzado de Datos con Pandas

# Elida Peña

**1.Instrucciones**

***Transformación de Datos**

**Crea nuevas columnas:** Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

**Clasifica los datos:** Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [1]:
#Importaciones de las librerías pandas, numpy y matplotlib
import pandas as pd

In [2]:
path = '../data/retail_sales_dataset.csv'
data_frame = pd.read_csv(path, sep=',', header=0)
data_frame.head(10)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200


In [3]:
# Crear la columna 'Net Income' asumiendo que 'Total Amount' es el ingreso bruto
data_frame['Net Income'] = data_frame['Total Amount'] - (data_frame['Total Amount'] * 0.2)  # Ejemplo: Restar un 20% de costos

# Normalizar la columna 'Price per Unit' para que esté entre 0 y 1
data_frame['Normalized Price'] = (data_frame['Price per Unit'] - data_frame['Price per Unit'].min()) / \
                                 (data_frame['Price per Unit'].max() - data_frame['Price per Unit'].min())

# Clasificar las ventas en 'High', 'Medium' y 'Low' basándonos en 'Total Amount'
def classify_sales(total):
    if total > 500:
        return 'High'
    elif 100 < total <= 500:
        return 'Medium'
    else:
        return 'Low'

data_frame['Sales Category'] = data_frame['Total Amount'].apply(classify_sales)

# Mostrar las primeras filas para verificar los cambios
print(data_frame[['Total Amount', 'Net Income', 'Normalized Price', 'Sales Category']].head())


   Total Amount  Net Income  Normalized Price Sales Category
0           150       120.0          0.052632         Medium
1          1000       800.0          1.000000           High
2            30        24.0          0.010526            Low
3           500       400.0          1.000000         Medium
4           100        80.0          0.052632            Low


**2.Agrupación y Agregación**

**Agrupación por múltiples columnas:** Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.

**Aplicar funciones de agregación:** Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [4]:
# Agrupar por 'Product Category' y 'Customer ID', y aplicar funciones de agregación
product_customer_aggregations = data_frame.groupby(['Product Category', 'Customer ID']).agg({
    'Total Amount': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Price per Unit': ['mean', 'min', 'max', 'std'],
    'Quantity': ['sum', 'mean']
})

# Mostrar los primeros resultados
print("Agrupaciones por Categoría de Producto y Cliente:")
print(product_customer_aggregations.head())

# Agrupar por 'Product Category' y 'Date' para obtener las ventas por mes y producto
# Convertir 'Date' a formato datetime si no lo está
data_frame['Date'] = pd.to_datetime(data_frame['Date'])

# Extraer el mes de la fecha
data_frame['Month'] = data_frame['Date'].dt.month

# Agrupar por 'Product Category' y 'Month'
product_month_aggregations = data_frame.groupby(['Product Category', 'Month']).agg({
    'Total Amount': ['sum', 'mean', 'count', 'min', 'max'],
    'Quantity': ['sum', 'mean']
})

# Mostrar los primeros resultados
print("\nAgrupaciones por Categoría de Producto y Mes:")
print(product_month_aggregations.head())


Agrupaciones por Categoría de Producto y Cliente:
                             Total Amount                                 \
                                      sum   mean count  min  max std var   
Product Category Customer ID                                               
Beauty           CUST001              150  150.0     1  150  150 NaN NaN   
                 CUST005              100  100.0     1  100  100 NaN NaN   
                 CUST006               30   30.0     1   30   30 NaN NaN   
                 CUST012               75   75.0     1   75   75 NaN NaN   
                 CUST021              500  500.0     1  500  500 NaN NaN   

                             Price per Unit               Quantity       
                                       mean  min  max std      sum mean  
Product Category Customer ID                                             
Beauty           CUST001               50.0   50   50 NaN        3  3.0  
                 CUST005               50.0  

**3.Análisis Personalizado con apply**

**Función personalizada:** Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.

**Ejemplo de uso avanzado:** Calcula la desviación de cada venta respecto a la media de su grupo.


In [5]:
# Calcular la desviación respecto a la media mensual de cada producto
def calculate_monthly_deviation(group):
    mean_total = group['Total Amount'].mean()
    group['Monthly Deviation from Mean'] = group['Total Amount'] - mean_total
    return group

# Aplicar la función personalizada usando groupby por 'Product Category' y 'Month'
data_frame = data_frame.groupby(['Product Category', 'Month']).apply(calculate_monthly_deviation)

# Mostrar algunas filas para verificar el resultado
print(data_frame[['Product Category', 'Month', 'Total Amount', 'Monthly Deviation from Mean']].head(10))

                           Product Category  Month  Total Amount  \
Product Category Month                                             
Beauty           1     20            Beauty      1           500   
                       31            Beauty      1            90   
                       188           Beauty      1            50   
                       210           Beauty      1          1500   
                       224           Beauty      1           100   
                       237           Beauty      1           500   
                       322           Beauty      1           900   
                       343           Beauty      1            30   
                       390           Beauty      1            50   
                       447           Beauty      1            60   

                            Monthly Deviation from Mean  
Product Category Month                                   
Beauty           1     20                    -35.769231  
         

C:\Users\59598\AppData\Local\Temp\ipykernel_15368\4288652666.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_frame = data_frame.groupby(['Product Category', 'Month']).apply(calculate_monthly_deviation)


**4.Documentación**

**Comentarios claros:** Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.

**Código legible:** Asegúrate de que el código sea legible y esté bien comentado.